<img style="float: right;"  src="images/LogoP.jpg" width="200">

# Demo : Test  BJT Transistor

Version 1.0 (20/6/2018)

This **notebook** can be used to test **BJT** transistors


## Import and connection

In [ ]:
# Import numpy
import numpy as np

# Import the main SLab module
import slab

In [ ]:
boardFolder = ''                                # Board folder (leave '' if you use only one board)
slab.setFilePrefix('../Files/')                 # Set File Prefix
slab.setCalPrefix('Calibrations/'+boardFolder)  # Set Calibration Prefix         
slab.connect()                                  # Connect to the board

In [ ]:
# Get the Vref value
vref = slab.gdata['vref']
vdd  = slab.gdata['vdd']

Execute the following **code cell** if you want the plots to be interactive.

In [ ]:
# Make plots interactive
slab.interactivePlots()
%matplotlib notebook

## Basic NPN connections

The following figure shows the basic connections to perform measurements on a **NPN** Device Under Test (DUT).

![Base NPN](images/Test_BJT/NPN_Base.png)

The NPN bipolar transistor with the base connected to **DAC1** enables us to have higher currents than the ones available on the **DAC** outputs. Note that we are using a **BC547** transistor that is limited to a maxim $100 mA$ collector current. This is ok because you cannot drain much more current from a typical **SLab hardware board**. If you want to test a transistor at higher currents, you will need to use a more complex setup.

The $R_b$ resistor is used to measure the **base** current. It will be selected for the maximum current we want to measure.

The $R_c$ resistor is used to measure the **collector** current. It will be selected for the maximum current we want to measure.

The **bias resistor** $R_{bias}$ is included to provide a proper bias to the NPN emitter follower when the **DUT** is draining no current. This resistor has no efect at all on the measurements so it can have any reasonable value as long as it is much bigger than $R_c$. In general $100 k\Omega$ is a good value.

## NPN Gummel and Beta plots

The [Gummel](https://en.wikipedia.org/wiki/Gummel_plot) plot shows the **base** and **collector** current as function of the $V_{be}$ voltage.

The same script also generates a **Beta** plot that shows $\beta(I_c)$

The $R_c$ resistor must be set so that we don't reach saturation at the minimum current.

$\qquad R_c = \frac{V_{DD}-V_{be}-V_{CE\:Sat\: Max}}{I_{c\: Max}}$

We will set the base current using **DAC2**. The $R_b$ resistor relates to the base current as:

$\qquad R_b = \frac{V_{DAC2\: Max}-V_{be}}{I_{b\:Max}}$

To execute the following **code cell** you need to setup the following parameters in the first lines:

* Value of $R_b$ used
* Value of $R_c$ used
* Maximum **DUT** saturation voltage
* Minimum **DAC2** voltage
* Maximum **DAC2** voltage
* **DAC2** voltage step

The maximum **DUT** saturation voltage is an upper bound to the saturation voltage. It is used to detect that we should stop the measurement because we are out of the **active** region. If you don't know this value, set an upper limit.

In [ ]:
# Parameters to set
Rc = 15          # [Ohm]
Rb = 1500        # [Ohm]
Vcesat = 1.0     # [V]
Vdac2min = 0.6   # [V]
Vdac2max = 2.5   # [V]
Vdac2inc = 0.02  # [V]

# V range
vdacR = np.arange(Vdac2min,Vdac2max,Vdac2inc)

# Initialize output
vbeR = []
ibR  = []
icR  = []

print('Measurement starts...')

# Perform measurement
slab.setVoltage(1,vref-0.1)
for vdac in vdacR:
    slab.setVoltage(2,vdac)           # Set DAC2 voltage
    slab.wait(0.1)
    vb = slab.readVoltage(2)          # Read base voltage
    vdac = slab.readVoltage(4)        # Read DAC2 voltage
    vrp = slab.readVoltage(1)         # Obtain Rc high node voltage
    vrn = slab.readVoltage(3)         # Obtain Rc low node voltage
    ib = (vdac-vb)/Rb                 # Calculate Ib (A)
    ic = (vrp-vrn)/Rc                 # Calculate Id (A)
    # Detect saturation
    if (vrn)<Vcesat:
        print('Reached saturation: Stop measuring')
        break
    vbeR.append(vb)    
    ibR.append(ib)        
    icR.append(ic)

# Set DACs to zero
slab.zero()    
    
# Show gummel plot
slab.plot1n(vbeR,[ibR,icR],'NPN Gummel Plot','Vbe (V)','Ib, Ic (A)',logy='True')

# Show beta plot
icR = np.array(icR)  # Convert to operate
ibR = np.array(ibR)
beta = icR/ibR
slab.plot11(icR*1000.0,beta,'Beta Plot','Ic (mA)','Beta')


## NPN Collector Current Curves

Draw the $I_c(V_{ce})$ curves for several $I_b$ values

This time we are not restricted to guarantee being in the **active** region, so the $R_c$ resistor just sets the current range you will get.

$\qquad I_c = \frac{V_{DD}-V_{be}-V_{CE}}{R_d}$

The current will be bounded by the limit at $V_{CE} = 0V$ give by:

$\qquad I_{c\: Max} = \frac{V_{DD}-V_{be}}{R_c}$

So, you can set Rd to be:

$\qquad R_c = \frac{V_{DD}-V_{be}}{I_{c\: Max}}$

There is a problem with the setup we are usuing because we are setting the **DAC2** voltage, not the base current. That means that base current will increase inside the saturation region. The code compensates this effect by scaling the **collector** current using information from the **base** current. You can eliminate this compensation by changing a flag at the start of the code.

To run the **code below** you need to set, on the first lines:

* Value of $R_b$ used
* Value of $R_c$ used
* Minimum **DAC2** voltage
* Maximum **DAC2** voltage
* **DAC2** voltage step

You can also change the **DAC1** sweep that sets the $V_{DS}$ voltage. That way you can change the resolution, for instance.

In [ ]:
# Parameters to set
Rc = 100        # [Ohm]
Rb = 100000      # [Ohm]
Vdac2min = 0.5   # [V]
Vdac2max = 2.6   # [V]
Vdac2inc = 1.0   # [V]

# Flag to use base current compensation
compensate_base_current = True

# Vdac1 sweep
Vdac1_min  = 0.5         # [V] 
Vdac1_max  = vref - 0.1  # [V]
Vdac1_step = 0.05         # [V]

# Set nreadings
slab.setDCreadings(400)

# Vdac2 range
vdac2R = np.arange(Vdac2min,Vdac2max,Vdac2inc)

# Initialize output lists
Vce_list  = []
Ib_list   = []
Ic_list   = []
labels    = []
for vdac2 in vdac2R:
    # Set Vdac2
    slab.setVoltage(2,vdac2)
    # Perform dcSweep
    data = slab.dcSweep(1,Vdac1_min,Vdac1_max,Vdac1_step)
    # Calculate Ib current (in mA)
    ib = 1000.0*(data[4]-data[2])/Rb
    # Calculate Ic current (in mA)
    ic = 1000.0*(data[1]-data[3])/Rc
    # Correct for non constant base current
    if compensate_base_current:
        ic = ic * ib[-1] / ib
    # Set label for this curve
    ib_ua = int(ib[-1]*10000.0)/10.0
    label = 'ib = ' + str(ib_ua) + ' uA'
    labels.append(label)
    # Add to list
    Vce_list.append(data[3])
    Ic_list.append(ic)
    Ib_list.append(ib)
    
# Set DACs to zero    
slab.zero()    
    
# Draw all curves
slab.plotnn(Vce_list,Ic_list,'Collector Current at several base currents'
                    ,'Vce (V)','Ic (mA)',labels)

You can use the following cell to disconnect from the board

In [ ]:
# Disconnect from the board
slab.disconnect()

## Document license

Copyright  ©  Vicente Jiménez (2018)  
This work is licensed under a Creative Common Attribution-ShareAlike 4.0 International license.  
This license is available at http://creativecommons.org/licenses/by-sa/4.0/

<img  src="images/cc_sa.png" width="200">